# Assignment 2
### Q1

For this task I have decided to use Tkinter as a tool for GUI.
It is rather easy to manipulate and offers a simple but efficient display.

In [ ]:
import tkinter as tk
from tkinter import messagebox, filedialog
import cv2 as cv
import numpy as np
import pickle
import processing_prints
import compare_prints
import evaluation
import os

class FingerprintRecognitionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Fingerprint Recognition System")

        self.enrolled_fingerprints = {}

        self.create_widgets()
        
        # Bind the closing event of the window to the on_closing method
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)

    def create_widgets(self):
       # Browse Frame
        browse_frame = tk.Frame(self.root, padx=10, pady=10)
        browse_frame.grid(row=0, column=0, columnspan=2, padx=10, pady=10)

        tk.Label(browse_frame, text="Select Fingerprint:").pack(side=tk.LEFT, padx=(0, 5))

        self.file_path_entry = tk.Entry(browse_frame, width=50)
        self.file_path_entry.pack(side=tk.LEFT, padx=(0, 5))

        self.browse_button = tk.Button(browse_frame, text="Browse", command=self.browse_file)
        self.browse_button.pack(side=tk.LEFT, padx=(0, 5))

        # Enrollment Frame
        enrollment_frame = tk.Frame(self.root, padx=10, pady=10)
        enrollment_frame.grid(row=1, column=0, padx=10, pady=10)

        tk.Label(enrollment_frame, text="Enroll Fingerprint:").grid(row=0, column=0, columnspan=2)

        self.enroll_button = tk.Button(enrollment_frame, text="Enroll", command=self.enroll_fingerprint)
        self.enroll_button.grid(row=1, column=0, columnspan=2, padx=5, pady=5)

        # Comparison Frame
        comparison_frame = tk.Frame(self.root, padx=10, pady=10)
        comparison_frame.grid(row=1, column=1, padx=10, pady=10)

        tk.Label(comparison_frame, text="Compare Fingerprint:").grid(row=0, column=0, columnspan=2)

        self.compare_button = tk.Button(comparison_frame, text="Compare", command=self.compare_fingerprint)
        self.compare_button.grid(row=1, column=0, padx=5, pady=5)

        # Evaluation Frame
        evaluation_frame = tk.Frame(self.root, padx=10, pady=10)
        evaluation_frame.grid(row=2, column=0, columnspan=2, padx=10, pady=10)

        tk.Label(evaluation_frame, text="Evaluation:").grid(row=0, column=0, columnspan=2)

        self.evaluate_button = tk.Button(evaluation_frame, text="Evaluate", command=self.evaluate_system)
        self.evaluate_button.grid(row=1, column=0, padx=5, pady=5)

    def browse_file(self):
        file_path = filedialog.askopenfilename()
        self.file_path_entry.delete(0, tk.END)
        self.file_path_entry.insert(0, file_path)

    def enroll_fingerprint(self):
        file_path = self.file_path_entry.get()
        if file_path:
            if file_path in self.enrolled_fingerprints:
                messagebox.showerror("Error", "This fingerprint has already been enrolled.")
            else:
                # Call the process_and_extract_features function to process the fingerprint
                fingerprint_image, valid_minutiae, local_structures = processing_prints.process_and_extract_features(file_path)

                # Store the features in a dictionary
                enrolled_features = {
                    'filename': os.path.basename(file_path),  # Store only the filename,  # Add file path to the dictionary
                    'fingerprint_image': fingerprint_image,
                    'valid_minutiae': valid_minutiae,
                    'local_structures': local_structures
                }

                # Save the enrolled features to the pickle file
                with open('enrolled_features.pickle', 'ab') as f:
                    pickle.dump(enrolled_features, f)

                # Add the enrolled fingerprint to the dictionary
                self.enrolled_fingerprints[file_path] = enrolled_features

                print("Enrolled features stored successfully!")
        else:
            messagebox.showerror("Error", "Please select a file for enrollment.")
                    
    def compare_fingerprint(self):
        file_path = self.file_path_entry.get()
        if file_path:
            # Load enrolled fingerprints from pickle file
            enrolled_fingerprints = []
            if os.path.exists('enrolled_features.pickle'):
                with open('enrolled_features.pickle', 'rb') as f:
                    while True:
                        try:
                            enrolled_features = pickle.load(f)
                            enrolled_fingerprints.append(enrolled_features)
                        except EOFError:
                            break
            
            # Process input fingerprint to extract features
            #f1, m1, ls1 = processing_prints.process_and_extract_features(file_path)

            # Compare input features with enrolled features
            compare_prints.compare_fingers(file_path, enrolled_fingerprints)
        else:
            messagebox.showerror("Error", "Please select a file for comparison.")

    def evaluate_system(self):
        threshold = 0.75
        
        file_path = self.file_path_entry.get()
        if file_path:
            # Load enrolled fingerprints from pickle file
            enrolled_fingerprints = []
            if os.path.exists('enrolled_features.pickle'):
                with open('enrolled_features.pickle', 'rb') as f:
                    while True:
                        try:
                            enrolled_features = pickle.load(f)
                            enrolled_fingerprints.append(enrolled_features)
                        except EOFError:
                            break
            
        cm, accuracy, precision, recall, f1 = evaluation.evaluate_system(file_path, enrolled_fingerprints, threshold)
        
        print(cm)
        
        messagebox.showinfo("Evaluate System", "System evaluation completed.")
        
    # Define a method to gracefully exit the application
    def on_closing(self):
        # Perform any cleanup tasks here...
        self.root.destroy()

def main():
    root = tk.Tk()
    app = FingerprintRecognitionApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()


In [ ]:
import pickle

# Load the data from the pickle file
with open('enrolled_features.pickle', 'rb') as f:
    enrolled_features = pickle.load(f)

# Print the content of the loaded data
print(enrolled_features)